In [37]:
import os
import re
import pandas as pd

In [42]:
book = pd.read_csv(
    f'{os.getcwd()}/data/french.csv',
    sep="|",
    header=None,
)
book.columns = ["sura", "ayat", "text"]
book['sura'] = book['sura'].astype(int)
book['ayat'] = book['ayat'].astype(int)

def cleaning_text(text):
    text = re.sub(r'` ', "'", text)
    text = re.sub(r' +(\.)', r'\1', text)
    return text

book["text"] = book["text"].apply(lambda x: cleaning_text(x))

In [43]:
book

,sura,ayat,text
0,1,1,"Au nom d´Allah, le Tout Miséricordieux, le Trè..."
1,1,2,"Louange à Allah, Seigneur de l´univers."
2,1,3,"Le Tout Miséricordieux, le Très Miséricordieux,"
3,1,4,Maître du Jour de la rétribution.
4,1,5,"C´est Toi [Seul] que nous adorons, et c´est To..."
...,...,...,...
6116,114,2,"Le Souverain des hommes,"
6117,114,3,"Dieu des hommes,"
6118,114,4,"contre le mal du mauvais conseiller, furtif,"
6119,114,5,"qui souffle le mal dans les poitrines des hommes,"
